In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], device_type='GPU')

2024-04-23 11:19:17.434391: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 11:19:17.473167: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 11:19:17.473718: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 11:19:18.061294: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-23 11:19:18.878421: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Do

In [2]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from tqdm.auto import tqdm

from typing import Any, Callable, Sequence, Union
import numpy as np
import scipy.stats as stats

import tensorflow as tf
tf.config.set_visible_devices([], device_type='GPU')

import jax
from jax import lax, random, numpy as jnp
import flax
from flax.core import freeze, unfreeze, FrozenDict
from flax import linen as nn
from flax import struct
from flax.training import train_state
from flax.training import orbax_utils

import optax
import orbax.checkpoint

from clu import metrics
from ml_collections import ConfigDict

from einops import reduce, rearrange
import wandb

from iqadatasets.datasets import *
from fxlayers.layers import *
from fxlayers.layers import GaborLayerLogSigma_, GaussianLayerGamma, FreqGaussianGamma, OrientGaussianGamma
from fxlayers.initializers import *
from JaxPlayground.utils.constraints import *
from JaxPlayground.utils.wandb import *

In [3]:
# dst_train = TID2008("/lustre/ific.uv.es/ml/uv075/Databases/IQA//TID/TID2008/", exclude_imgs=[25])
# dst_val = TID2013("/lustre/ific.uv.es/ml/uv075/Databases/IQA//TID/TID2013/", exclude_imgs=[25])
# dst_train = TID2008("/media/disk/databases/BBDD_video_image/Image_Quality//TID/TID2008/", exclude_imgs=[25])
# dst_val = TID2013("/media/disk/databases/BBDD_video_image/Image_Quality//TID/TID2013/", exclude_imgs=[25])
# dst = KADIK10K("/media/disk/databases/BBDD_video_image/Image_Quality/KADIK10K/")
# dst = PIPAL("/media/disk/databases/BBDD_video_image/Image_Quality/PIPAL/")
# dst = KADIK10K("/lustre/ific.uv.es/ml/uv075/Databases/IQA/KADIK10K/")
dst = KADIK10K("/media/databases/IQA/KADIK10K/")

In [4]:
img, img_dist, mos = next(iter(dst.dataset))
img.shape, img_dist.shape, mos.shape

(TensorShape([384, 512, 3]), TensorShape([384, 512, 3]), TensorShape([]))

In [5]:
id = "7zqnh75q" # VerySimple (KLD)

In [6]:
api = wandb.Api()
prev_run = api.run(f"jorgvt/PerceptNet_KLD/{id}")

In [7]:
config = ConfigDict(prev_run.config["_fields"])
config

BATCH_SIZE: 32
EPOCHS: 30
LEARNING_RATE: 0.0003
SEED: 42

In [8]:
for file in prev_run.files():
    file.download(root=prev_run.dir, replace=True)

In [9]:
wandb.init(project="PerceptNet_KLD",
           name=prev_run.name,
           job_type="evaluate",
           mode="online",
           )
config = config
config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jorgvt. Use `wandb login --relogin` to force relogin


BATCH_SIZE: 32
EPOCHS: 30
LEARNING_RATE: 0.0003
SEED: 42

In [29]:
def resize(img, img_dist, mos):
    h, w = 384, 512
    img = tf.image.resize(img, (h//8, w//8))
    img_dist = tf.image.resize(img_dist, (h//8, w//8))
    return img, img_dist, mos

In [30]:
dst_rdy = dst.dataset.batch(config.BATCH_SIZE, num_parallel_calls=tf.data.AUTOTUNE)\
                     .map(resize)

## Define the model we're going to use

> It's going to be a very simple model just for demonstration purposes.

In [ ]:
# class PerceptNet(nn.Module):
#     """IQA model inspired by the visual system."""

#     @nn.compact
#     def __call__(self,
#                  inputs,
#                  **kwargs,
#                  ):
#         outputs = rearrange(inputs, "b h w c -> b (h w c)")
#         if args["kld"]:
#             mean = nn.Dense(features=2)(outputs)
#             std = nn.Dense(features=2)(outputs)
#             return mean, std
#         elif args["mse"]:
#             return nn.Dense(features=2)(outputs)

In [11]:
class PerceptNet(nn.Module):
    """IQA model inspired by the visual system."""

    @nn.compact
    def __call__(self,
                 inputs,
                 **kwargs,
                 ):
        outputs = rearrange(inputs, "b h w c -> b (h w c)")
        mean = nn.Dense(features=2)(outputs)
        std = nn.Dense(features=2)(outputs)
        return mean, std

## Define the metrics with `clu`

In [12]:
@struct.dataclass
class Metrics(metrics.Collection):
    """Collection of metrics to be tracked during training."""
    loss: metrics.Average.from_output("loss")

By default, `TrainState` doesn't include metrics, but it's very easy to subclass it so that it does:

In [13]:
class TrainState(train_state.TrainState):
    metrics: Metrics
    state: FrozenDict

We'll define a function that initializes the `TrainState` from a module, a rng key and some optimizer:

In [14]:
def create_train_state(module, key, tx, input_shape):
    """Creates the initial `TrainState`."""
    variables = module.init(key, jnp.ones(input_shape))
    state, params = variables.pop('params')
    return TrainState.create(
        apply_fn=module.apply,
        params=params,
        state=state,
        tx=tx,
        metrics=Metrics.empty()
    )

## Define evaluation step

In [15]:
def kld(mean_p, std_p, mean_q, std_q, axis=(1)):
    """Assume diagonal covariance matrix and that the input is the logvariance."""
    std_p, std_q = jnp.exp(std_p), jnp.exp(std_q)
    def safe_div(a, b): return a/b #jnp.where(a == b, 1, a/b)
    det_p = jnp.prod(std_p, axis=axis) + 1e-5
    det_q = jnp.prod(std_q, axis=axis) + 1e-5
    
    return jnp.log(safe_div(det_p, det_q)) + jnp.sum((1/std_q)*(mean_p - mean_q)**2, axis=axis) + jnp.sum(std_p/std_q, axis=axis)

In [16]:
@jax.jit
def compute_distance(*, state, batch):
    """Obtaining the metrics for a given batch."""
    img, img_dist, mos = batch
    
    (img_mean, img_std), updated_state = state.apply_fn({"params": state.params, **state.state}, img, mutable=list(state.state.keys()), train=False)
    (img_dist_mean, img_dist_std), updated_state = state.apply_fn({"params": state.params, **state.state}, img_dist, mutable=list(state.state.keys()), train=False)

    ## Calculate the KLD
    dist = kld(img_mean, img_std, img_dist_mean, img_dist_std)
    
    return dist

## Load the pretrained model!

In [17]:
state = create_train_state(PerceptNet(), random.PRNGKey(config.SEED), optax.adam(config.LEARNING_RATE), input_shape=(1,384,512,3))

2024-04-23 11:22:05.802294: W external/xla/xla/service/gpu/nvptx_compiler.cc:596] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.52). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [18]:
def check_trainable(path):
    return False
    # return ("A" in path) or ("alpha_achrom" in path) or ("alpha_chrom_rg" in path) or ("alpha_chrom_yb" in path)

In [19]:
trainable_tree = freeze(flax.traverse_util.path_aware_map(lambda path, v: "non_trainable" if check_trainable(path)  else "trainable", state.params))

In [20]:
param_count = sum(x.size for x in jax.tree_util.tree_leaves(state.params))
trainable_param_count = sum([w.size if t=="trainable" else 0 for w, t in zip(jax.tree_util.tree_leaves(state.params), jax.tree_util.tree_leaves(trainable_tree))])
param_count, trainable_param_count

(2359300, 2359300)

In [21]:
wandb.run.summary["total_parameters"] = param_count
wandb.run.summary["trainable_parameters"] = trainable_param_count

In [22]:
optimizers = {
    "trainable": optax.adam(learning_rate=config.LEARNING_RATE),
    "non_trainable": optax.set_to_zero(),
}

In [23]:
tx = optax.multi_transform(optimizers, trainable_tree)

In [24]:
state = create_train_state(PerceptNet(), random.PRNGKey(config.SEED), tx, input_shape=(1,384,512,3))

In [25]:
# Before actually training the model we're going to set up the checkpointer to be able to save our trained models:
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
save_args = orbax_utils.save_args_from_target(state)

In [26]:
# Load weights
state = orbax_checkpointer.restore(os.path.join(prev_run.dir,"model-best"), item=state)

## Evaluate!

In [31]:
metrics_history = {
    "distance": [],
    "mos": [],
}

In [34]:
%%time
for batch in tqdm(dst_rdy.as_numpy_iterator()):
    img, img_dist, mos = batch
    distance = compute_distance(state=state, batch=batch)
    metrics_history["distance"].extend(distance)
    metrics_history["mos"].extend(mos)
    # break

0it [00:00, ?it/s]

2024-04-23 11:24:07.577815: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: /media/databases/IQA/KADIK10K/images/I23_07_01.png; No such file or directory


NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} /media/databases/IQA/KADIK10K/images/I23_07_01.png; No such file or directory
	 [[{{node ReadFile_1}}]] [Op:IteratorGetNext] name: 

In [ ]:
assert len(metrics_history["distance"]) == len(dst.data)

In [33]:
stats.pearsonr(metrics_history["distance"], metrics_history["mos"]), stats.spearmanr(metrics_history["distance"], metrics_history["mos"])

(PearsonRResult(statistic=-0.09987253410685878, pvalue=1.5208760413125765e-07),
 SignificanceResult(statistic=-0.09445223820020526, pvalue=6.913891870512814e-07))

In [6]:
results = dst.data.copy()
results["Distance"] = metrics_history["distance"]
results.head()

,dist_img,ref_img,dmos,var,Distance
0,I01_01_01.png,I01.png,4.57,0.496,0.0
1,I01_01_02.png,I01.png,4.33,0.869,30.293993
2,I01_01_03.png,I01.png,2.67,0.789,72.83459
3,I01_01_04.png,I01.png,1.67,0.596,124.22675
4,I01_01_05.png,I01.png,1.10,0.300,224.45325


In [ ]:
wandb.log({"KADID10K": wandb.Table(dataframe=results),
           "KADID10K_pearson": stats.pearsonr(metrics_history["distance"], metrics_history["mos"])[0],
           "KADID10K_spearman": stats.spearmanr(metrics_history["distance"], metrics_history["mos"])[0],
           })

<class 'wandb.errors.Error'>: You must call wandb.init() before wandb.log()

In [35]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


total_parameters,2359300
trainable_parameters,2359300
